In [60]:
from astropy import units as u
from astroquery.atomic import AtomicLineList
import astroquery.atomic as a
from astropy.table import conf

# Lines around the neutral 8152 line

There are certainly some He lines there.

In [2]:
wavelength_range = (8152 * u.Angstrom, 70 * u.Angstrom)
res = AtomicLineList.query_object(
    wavelength_range=wavelength_range,
    wavelength_type='Air',
    minimal_abundance="6",
    depl_factor="0",
    element_spectrum="\n".join(['H', 'He']),
    #get_query_payload=True,
)

/Users/will/miniconda3/envs/py39/lib/python3.9/site-packages/astroquery/atomic/core.py:309: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 309 of the file /Users/will/miniconda3/envs/py39/lib/python3.9/site-packages/astroquery/atomic/core.py. To get rid of this warning, pass the additional argument 'features="html5lib"' to the BeautifulSoup constructor.

  bs = BeautifulSoup(response.text, 'html5')
/Users/will/miniconda3/envs/py39/lib/python3.9/site-packages/astroquery/atomic/core.py:249: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system

In [3]:
res

LAMBDA AIR ANG,SPECTRUM,TT,CONFIGURATION,TERM,J J,A_ki,LEVEL ENERGY CM 1
float64,str4,str2,str12,str6,str3,float64,str23
8093.08,He I,E1,1s.3p-1s.17s,3Po-3S,2-1,29100.0,185564.67 - 197917.50
8093.09,He I,E1,1s.3p-1s.17s,3Po-3S,1-1,17500.0,185564.69 - 197917.50
8093.27,He I,E1,1s.3p-1s.17s,3Po-3S,0-1,5830.0,185564.96 - 197917.50
8094.08,He I,E1,1s.3s-1s.10p,1S-1Po,0-1,144000.0,184864.93 - 197216.24
8127.09,He I,E1,1s.3p-1s.16s,3Po-3S,2-1,35100.0,185564.67 - 197865.81
8127.11,He I,E1,1s.3p-1s.16s,3Po-3S,1-1,21100.0,185564.69 - 197865.81
8127.29,He I,E1,1s.3p-1s.16s,3Po-3S,0-1,7030.0,185564.96 - 197865.81
8155.59,He I,E1,1s.3p-1s.15d,3Po-3D,2-*,68500.0,185564.67 - 197822.82
8155.61,He I,E1,1s.3p-1s.15d,3Po-3D,1-*,41100.0,185564.69 - 197822.82


In [66]:
wavelength_range = (8152 * u.Angstrom, 70 * u.Angstrom)
ionlist = " I-IV\n".join(
    [
        "C", 
        "N", 
        "O", 
        "S",
        "Si",
        "Cl",
        "Fl",
        "Fe",
        "Ne",
        "Ar",
        "Ni",
        "Ca",
        "Mg",
    ]
)
res = AtomicLineList.query_object(
    wavelength_range=wavelength_range,
    wavelength_type='Air',
    lower_level_energy_range=3 * u.eV,
    #transitions=aa.Transition.nebular,
    element_spectrum=ionlist,
    #get_query_payload=True,
)

In [67]:
res

LAMBDA AIR ANG,SPECTRUM,TT,CONFIGURATION,TERM,J J,A_ki,LEVEL ENERGY CM 1
float64,str7,str2,str33,str8,str8,str4,str22
8105.651,Ni I],E1,3d8.(3P).4s2-3d8.(3F).4s.4p.(3Po),3P-5Do,2-1,None,15609.84 - 27943.52
8108.3199,Fe I],E1,3d7.(2G).4s-3d7.(4F).4p,a3G-y5Fo,4-3,None,21999.13 - 34328.75
8111.975,[Ni I],M1,3d9.(2D).4s-3d8.(3P).4s2,1D-3P,2-1,None,3409.94 - 15734.00
8112.1609,Fe I],E1,3d7.(2G).4s-3d7.(4F).4p,a3G-y5Fo,5-4,None,21715.73 - 34039.51
8119.142,[Fe II],M1,3d7-3d6.(3P4).4s,a4P-b2P,5/2-3/2,None,13474.41 - 25787.60
8129.3131,Fe I],E1,3d7.(2G).4s-3d7.(4F).4p,a3G-y5Fo,3-2,None,22249.43 - 34547.21
8151.3424,[Fe I],M1,3d6.4s2-3d7.(4F).4s,a5D-a3F,2-2,None,704.01 - 12968.55
8180.0809,Fe I],E1,3d6.4s2-3d7.(4F).4p,b3F-y5Do,3-4,None,20874.48 - 33095.94
8184.503,[Fe II],E2,3d6.(3H).4s-3d6.(1G4).4s,a4H-c2G,13/2-9/2,None,21251.61 - 33466.46


In [68]:
wavelength_range = (8152 * u.Angstrom, 70 * u.Angstrom)
ionlist = "\n".join([
    "N I", "N II", 
    "C I", "C II", 
    "O I", "O II", 
    "Si I", "Si II", 
    "Ca I", "Ca II", 
])

res = AtomicLineList.query_object(
    wavelength_range=wavelength_range,
    wavelength_type='Air',
    #lower_level_energy_range=3 * u.eV,
    #transitions=aa.Transition.nebular,
    element_spectrum=ionlist,
    #get_query_payload=True,
)
res.show_in_notebook()

idx,LAMBDA AIR ANG,SPECTRUM,TT,CONFIGURATION,TERM,J J,A_ki,LEVEL ENERGY CM 1
0,8083.8,C I,E1,2s2.2p.(2Po).3p-2s2.2p.(2Po).5s,3P-3Po,2-1,6.08E+05,71385.38 - 83752.41
1,8084.352,Si I,E1,3s2.3p.3d-3s2.3p.(2Po<3/2>).7p<3/2>,"3Fo-(3/2,3/2)",4-3,None,50054.81 - 62420.99
2,8085.19,Ca I,E1,4s.5p-4s.24d,1Po-1D,1-2,None,36731.61 - 49096.52
3,8086.17,Ca I,E1,3d.4p-4s.11d,3Fo-3D,4-3,2.87E+06,35896.89 - 48260.29
4,8088.02,Ca I,E1,4s.5p-4s.25s,1Po-1S,1-0,None,36731.61 - 49092.19
5,8088.92,Ca I,E1,4s.5p-4s.18d,3Po-3D,2-3,None,36575.12 - 48934.32
6,8089.06,Ca I,E1,4s.5p-4s.18d,3Po-3D,2-1,None,36575.12 - 48934.10
7,8089.08,N II],E1,2s2.2p.(2Po).3p-2s2.2p.(2Po).3d,1D-3Fo,2-3,None,174211.83 - 186570.78
8,8089.1,Ca I,E1,4s.5p-4s.18d,3Po-3D,2-2,None,36575.12 - 48934.04
9,8089.19,Ca I],E1,4s.5p-4s.25s,1Po-3S,1-1,None,36731.61 - 49090.40


# All the [Fe I] lines (and others)

First look for all the neutral lines of common elements that fall in the wavelength range where we see the lines.

Start off with just the forbidden lines, so we do not get too many. Also, impose an upper energy level cutoff of less than 4 eV, which would make sense if they wer excited collisionally.

In [99]:
wavelength_range = (
    8100 * u.Angstrom, 9200 * u.Angstrom
)
elements = "\n".join(
    [
        "C", "N", "O",
        "Mg", "Ca", "Na", 
        "S", "Si", "P",
        "F", "Cl",
        "Ar", "Xe",
        "Fe", "Ni", "Co",
    ]
)
res = AtomicLineList.query_object(
    wavelength_range=wavelength_range,
    wavelength_type='Air',
    upper_level_energy_range=10 * u.eV,
    element_spectrum=elements,
    transitions=aa.Transition.nebular,
)

/Users/will/miniconda3/envs/py39/lib/python3.9/site-packages/astroquery/atomic/core.py:249: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 249 of the file /Users/will/miniconda3/envs/py39/lib/python3.9/site-packages/astroquery/atomic/core.py. To get rid of this warning, pass the additional argument 'features="html5lib"' to the BeautifulSoup constructor.

  data = StringIO(BeautifulSoup(response.text, 'html5').find('pre').text.strip())


Add a new column that gives the ion stage: I, II, III, IV, etc

In [100]:
res["stage"] = [s.strip("[]").split()[-1] for s in res["SPECTRUM"]]

And use it to selecct just the neutral ions, since the API does not allow this to be done on the query. 

In [101]:
res[res["stage"]=="I"].show_in_notebook()

idx,LAMBDA AIR ANG,SPECTRUM,TT,CONFIGURATION,TERM,J J,A_ki,LEVEL ENERGY CM 1,stage
0,8109.43,[Co I],E2,3d8.(3F).4s-3d8.(1D).4s,b4F-a2D,7/2-3/2,None,4142.66 - 16470.60,I
1,8111.2,[Co I],M1,3d8.(3F).4s-3d7.4s2,b4F-a2G,7/2-9/2,None,4142.66 - 16467.90,I
2,8111.975,[Ni I],M1,3d9.(2D).4s-3d8.(3P).4s2,1D-3P,2-1,None,3409.94 - 15734.00,I
3,8151.3424,[Fe I],M1,3d6.4s2-3d7.(4F).4s,a5D-a3F,2-2,None,704.01 - 12968.55,I
4,8176.4,[Co I],E2,3d7.4s2-3d7.4s2,a4F-a4P,3/2-3/2,None,1809.33 - 14036.28,I
5,8194.531,[Ni I],M1,3d9.(2D).4s-3d8.(3P).4s2,1D-3P,2-2,None,3409.94 - 15609.84,I
6,8201.74,[Ni I],M1,3d8.(3F).4s2-3d8.(1D).4s2,3F-1D,3-2,None,1332.16 - 13521.35,I
7,8222.88,[Co I],E2,3d8.(3F).4s-3d7.4s2,b4F-a2G,3/2-7/2,None,5075.83 - 17233.68,I
8,8231.5783,[Fe I],M1,3d6.4s2-3d7.(4F).4s,a5D-a3F,3-3,None,415.93 - 12560.93,I
9,8270.41,[Co I],M1,3d8.(3F).4s-3d8.(1D).4s,b4F-a2D,5/2-5/2,None,4690.18 - 16778.16,I


So there are some partial overlaps with the observed wavelengths here. 

For instance, 9020 might be [Co I] 9019.65

But most of the wavelengths are misses.



In [106]:
(1 * u.eV).to(1/u.cm, equivalencies=u.equivalencies.spectral())

<Quantity 8065.54393735 1 / cm>

Next thing to try is to expand search to include permitted lines as well as forbidden ones. If they are excited by fluorescence of recombination, then they should have excitation energies less than the ionization potential of the neutral atom, which for these elements is about 6 eV.  

In [109]:
res = AtomicLineList.query_object(
    wavelength_range=wavelength_range,
    wavelength_type='Air',
    upper_level_energy_range=7 * u.eV,
    element_spectrum="Fe",
    #transitions=aa.Transition.nebular,
)
res["stage"] = [s.strip("[]").split()[-1] for s in res["SPECTRUM"]]

/Users/will/miniconda3/envs/py39/lib/python3.9/site-packages/astroquery/atomic/core.py:249: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 249 of the file /Users/will/miniconda3/envs/py39/lib/python3.9/site-packages/astroquery/atomic/core.py. To get rid of this warning, pass the additional argument 'features="html5lib"' to the BeautifulSoup constructor.

  data = StringIO(BeautifulSoup(response.text, 'html5').find('pre').text.strip())


In [110]:
res[res["stage"]=="I"].show_in_notebook()

idx,LAMBDA AIR ANG,SPECTRUM,TT,CONFIGURATION,TERM,J J,A_ki,LEVEL ENERGY CM 1,stage
0,8104.5764,Fe I],E1,3d7.(4F).4p-3d6.4s.(6D).4d,y3Do-e7F,1-2,None,38995.73 - 51331.05,I
1,8106.6971,Fe I,E1,3d6.(3P).4s.4p.(3Po)-3d6.4s.(6D).6s,x5Po-5D,3-3,None,42532.74 - 54864.83,I
2,8108.3199,Fe I],E1,3d7.(2G).4s-3d7.(4F).4p,a3G-y5Fo,4-3,None,21999.13 - 34328.75,I
3,8112.1609,Fe I],E1,3d7.(2G).4s-3d7.(4F).4p,a3G-y5Fo,5-4,None,21715.73 - 34039.51,I
4,8114.2528,Fe I],E1,3d7.(4F).4p-3d6.4s.(6D).4d,y3Do-f7D,2-2,None,38678.04 - 50998.64,I
5,8114.9121,Fe I],E1,3d6.4s.(6D).5s-3d7.(a2D).4p,e7D-w1Do,3-2,None,43434.62 - 55754.23,I
6,8115.9991,Fe I],E1,3d7.(2P).4s-3d6.(5D).4s.4p.(1Po),c3P-y5Po,0-1,None,25091.60 - 37409.55,I
7,8118.2102,Fe I,E1,3d6.(3F).4s.4p.(3Po)-3d6.4s.(6D).5d,v5Do-5D,4-3,None,44022.52 - 56337.12,I
8,8119.2717,Fe I,E1,3d6.(5D).4s.4p.(1Po)-3d7.(4F).4d,x5Fo-h5D,3-4,None,40842.15 - 53155.14,I
9,8120.257,Fe I,E1,3d6.(3P).4s.4p.(3Po)-3d6.4s.(6D).6s,x5Po-5D,1-0,None,43079.02 - 55390.52,I


# Look at the supposed Ca I lines

## The 7890 line

This looks a lot like the [Fe III] lines

In [17]:
wavelength_range = (7890 * u.Angstrom, 0.2 * u.Angstrom)
#wavelength_range = (6500 * u.Angstrom, 6600 * u.Angstrom)
#AtomicLineList.FORM_URL = "https://www.pa.uky.edu/~peter/newpage/"
res = AtomicLineList.query_object(
    wavelength_range=wavelength_range,
    wavelength_type='Air',
    minimal_abundance="6",
    depl_factor="0",
    #element_spectrum='H I',
    #get_query_payload=True,
)

In [18]:
res

LAMBDA AIR ANG,SPECTRUM,TT,CONFIGURATION,TERM,J J,A_ki,LEVEL ENERGY CM 1
str8,str8,str2,str36,str18,str7,str8,str25
7608.,Si VII],E1,2s2.2p3.(2Po).3d-2s.2p4.(4P).3s,1Po-3P,1-0,None,1491287.00 - 1504430.00
7698.,Ar X],E1,2s2.2p4.(1S).3p-2s2.2p4.(3P).3d,2Po-4D,1/2-3/2,None,2555193.00 - 2568152.00
7788.,Si XI,E1,2p.4p-2p.4d,3P-3Do,2-2,1.13E+06,3187600.00 - 3200430.00
7838.,Si XI,E1,2p.4d-2p.4f,3Fo-3F,2-3,3.26E+04,3188940.00 - 3201700.00
7868.,Si XI,E1,2p.4d-2p.4f,3Fo-3F,2-2,3.61E+05,3188940.00 - 3201650.00
7873.,N VI,E1,1s.8d-1s.9p,3D-3Po,*-*,2.63E+07,4390976.00 - 4403673.70
7878.,Si XI,E1,2p.4d-2p.4f,3Do-3D,2-1,1.59E+05,3200430.00 - 3213120.00
7888.,Fe III],E1,3d5.(b2G).4p-3d5.(4G).4d,3Fo-5F,2-2,None,167002.00 - 179676.89
7889.9,N II],E1,2s2.2p.(2Po).6p-2s2.2p.(2Po).15d,1P-3Po,1-2,None,224286.63 - 236957.50


So it looks like the [Ni III] 7889.9 line is a very good candidate.  It is a nebular forbidden line, so would be expected to be strong. 

In [8]:
AtomicLineList.query_object_async??

Signature:
AtomicLineList.query_object_async(
    wavelength_range=None,
    wavelength_type='',
    wavelength_accuracy=None,
    element_spectrum=None,
    minimal_abundance=None,
    depl_factor=None,
    lower_level_energy_range=None,
    upper_level_energy_range=None,
    nmax=None,
    multiplet=None,
    transitions=None,
    show_fine_structure=None,
    show_auto_ionizing_transitions=None,
    output_columns=('spec', 'type', 'conf', 'term', 'angm', 'prob', 'ener'),
    cache=True,
    get_query_payload=False,
)
Docstring:
Queries Atomic Line List for the given parameters adnd returns the
result as a `~astropy.table.Table`. All parameters are optional.

Parameters
----------
wavelength_range : pair of `astropy.units.Unit` values
    Wavelength range. Can be done in two ways: supply a lower and
    upper limit for the region or, supply the central wavelength and
    the 1 sigma error (68 % confidence value) for that line. If the
    first number is smaller than the second number

In [14]:
aa.Transition.nebular

AtomicTransition<'Neb'>

In [15]:
simple_transitions = [aa.Transition.all, aa.Transition.nebular]

In [16]:
simple_transitions

[AtomicTransition<'All'>, AtomicTransition<'Neb'>]

In [17]:
"Neb" in simple_transitions

False

In [18]:
aa.utils.is_valid_transitions_param("Neb")

False

In [23]:
aa.utils.is_valid_transitions_param(aa.Transition.nebular)

True

In [37]:
from astropy import units as u


In [41]:
u.spectral??

Signature: u.spectral()
Source:   
def spectral():
    """
    Returns a list of equivalence pairs that handle spectral
    wavelength, wave number, frequency, and energy equivalencies.

    Allows conversions between wavelength units, wave number units,
    frequency units, and energy units as they relate to light.

    There are two types of wave number:

        * spectroscopic - :math:`1 / \\lambda` (per meter)
        * angular - :math:`2 \\pi / \\lambda` (radian per meter)

    """
    hc = _si.h.value * _si.c.value
    two_pi = 2.0 * np.pi
    inv_m_spec = si.m ** -1
    inv_m_ang = si.radian / si.m

    return Equivalency([
        (si.m, si.Hz, lambda x: _si.c.value / x),
        (si.m, si.J, lambda x: hc / x),
        (si.Hz, si.J, lambda x: _si.h.value * x, lambda x: x / _si.h.value),
        (si.m, inv_m_spec, lambda x: 1.0 / x),
        (si.Hz, inv_m_spec, lambda x: x / _si.c.value,
         lambda x: _si.c.value * x),
        (si.J, inv_m_spec, lambda x: x / hc, lambda x:

In [43]:
u.Equivalency?

Init signature: u.Equivalency(equiv_list, name='', kwargs=None)
Docstring:     
A container for a units equivalency.

Attributes
----------
name: `str`
    The name of the equivalency.
kwargs: `dict`
    Any positional or keyword arguments used to make the equivalency.
File:           ~/miniconda3/envs/py39/lib/python3.9/site-packages/astropy/units/equivalencies.py
Type:           ABCMeta
Subclasses:     
